### Embeddings

In [2]:
#pip install typing-extensions --upgrade

In [3]:
#!pip install -U sentence-transformers

In [11]:
!pip install datasets

     |████████████████████████████████| 493 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 26.8 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 369 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 12.8 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 15.8 MB/s eta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.3
    Uninstalling dill-0.3.3:
      Successfully uninstalled dill-0.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that a

In [12]:
!pip install gensim

     |████████████████████████████████| 24.1 MB 5.1 MB/s eta 0:00:01
  Using cached scipy-1.10.1-cp38-cp38-macosx_10_9_x86_64.whl (35.0 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.6.2
    Uninstalling scipy-1.6.2:
      Successfully uninstalled scipy-1.6.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.1.0 requires scipy<1.10,>=1.4.1, but you have scipy 1.10.1 which is incompatible.
ydata-profiling 4.1.0 requires tqdm<4.65,>=4.48.2, but you have tqdm 4.66.1 which is incompatible.


In [4]:
from sentence_transformers import SentenceTransformer,util

model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = ['el gato juega afuera',
             'me gusta tocar guitarra',
             'me encanta la pasta',
             'esta pelicula estuvo asombrosa',
             'el perro esta afuera',
             'te gusta la musica?',
             'estuvo muy buena la pelicula',
             'te gustaria una pizza?']
embeddings = model.encode(sentences)

In [6]:
embeddings.shape

(8, 384)

#### Comparar frases con modelo de embeddings

In [9]:
cosine_scores= util.cos_sim(embeddings,embeddings)

pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1,len(cosine_scores)):
        pairs.append({'index':[i,j],'score':cosine_scores[i][j]})

pairs = sorted(pairs,key=lambda x: x['score'],reverse=True)
for pairs in pairs[0:5]:
    i,j = pairs['index']
    print("{}---{}".format(sentences[i],sentences[j]))

esta pelicula estuvo asombrosa---estuvo muy buena la pelicula
te gusta la musica?---te gustaria una pizza?
el gato juega afuera---el perro esta afuera
me gusta tocar guitarra---te gusta la musica?
me encanta la pasta---te gustaria una pizza?


In [8]:
cosine_scores

tensor([[1.0000, 0.4441, 0.4348, 0.4453, 0.6610, 0.5136, 0.4515, 0.4513],
        [0.4441, 1.0000, 0.4452, 0.3903, 0.4293, 0.6582, 0.3939, 0.5471],
        [0.4348, 0.4452, 1.0000, 0.5131, 0.4545, 0.5705, 0.5273, 0.6540],
        [0.4453, 0.3903, 0.5131, 1.0000, 0.6188, 0.4869, 0.7083, 0.5322],
        [0.6610, 0.4293, 0.4545, 0.6188, 1.0000, 0.4857, 0.5354, 0.4627],
        [0.5136, 0.6582, 0.5705, 0.4869, 0.4857, 1.0000, 0.5080, 0.6938],
        [0.4515, 0.3939, 0.5273, 0.7083, 0.5354, 0.5080, 1.0000, 0.5305],
        [0.4513, 0.5471, 0.6540, 0.5322, 0.4627, 0.6938, 0.5305, 1.0000]])

#### Embeddings con Gensim

In [15]:
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation,strip_numeric,strip_short,stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datasets import load_dataset
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.tokenize import word_tokenize


In [16]:
dataset_corpues = load_dataset("large_spanish_corpus","ParaCrawl")

KeyboardInterrupt: 

In [ ]:
subset = dataset_corpues['train'].select(range(1000000))

##### Preprocesamiento

In [ ]:
def clean_text(sentence_batch):
    text_list = sentence_batch['text']
    cleaned_text_list = []
    
    for text in text_list:
        # convertir a minusculas
        text = text.lower
        # eliminar urls
        text = re.sub(r"http\S+|www\S+|https\S+","",text,flags=re.MULTILINE)
        # elimina menciones @ #
        text = re.sub(r"\@\w+|\#\w+","",text)
        #elimina caracteres de puntuacion
        text = strip_punctuation(text)
        #elimina numeros
        text = strip_numeric(text)
        #elimina palabras cortas
        text = strip_short(text,minsize=2)
        #elimina stopwords
        stop_words = set(stopwords.words("spanish"))
        word_tokens = word_tokenize(text)

        filtered_text = [x for x in word_tokens if x not in stop_words]
        cleaned_text_list.append(filtered_text)
    return {'text':cleaned_text_list}

In [ ]:

sentence_corpus = subset.map(clean_text,batched=True)

#### Carga y uso de modelo de embeddings Word2Vec

In [ ]:
model = Word2Vec(sentence_corpus['text'],vector_size=100,windows=5,min_counts=2,workers=6,sg=1)
model.save("word2.vec.model")

In [ ]:
model.wv['rey']

In [ ]:
model.wv.most_similar(['comida'],topn=3)

In [ ]:
word_vectors = model.wv
vectors = word_vectors.vectors
words = word_vectors.index_to_key

#### Almacenamiento de embeddings

In [ ]:
df_vectors = pd.DataFrame(vectors)
df_vectors.to_csv('embeddings.tsv',sep='\t',index=False)

In [ ]:
df_words = pd.DataFrame(words)
df_words.to_csv('labels.tsv',sep='\t',index=False)